# Quantum Key Distribution (QKD): 
## E91 protocol implementation

Developed in 1991 by Artur Ekert, the E91 protocol is a form of Quantum Key Distribution (QKD) which leverages quantum entanglement to securly quantum channel between two parties, Alice and Bob, leveraging the uncertainty principle and the no-cloning theorem from quantum mechanics. This prevents any potential eavesdropper, such as Eve, from intercepting the key without detection.

In [4]:
# import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute,IBMQ
from qiskit_ibm_provider import IBMProvider

In [102]:
api_token = ""

### Set up IBM account

In [6]:
IBMQ.enable_account(api_token)
provider = IBMQ.get_provider(hub='ibm-q')
print(provider.backends())
backend = provider.get_backend('ibmq_qasm_simulator')


/tmp/ipykernel_1338751/1614407315.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.enable_account(api_token)
/tmp/ipykernel_1338751/1614407315.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.enable_account(api_token)


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_brisbane') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_kyoto') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_osaka') from IBMQ(hub='ibm-q', group='open', project='main')>]


### Set up quantum circuit

In [89]:
#Set up quantum circuit
def set_up_qc(qubits):
    q = QuantumRegister(qubits, 'q')
    c = ClassicalRegister(qubits, 'c')
    qc = QuantumCircuit(q, c)
    return qc, q, c

### Prepare entangled qubits
A centeral source prepares pairs of entangled qubits which are sent to Alice and Bob

In [90]:
def generate_entangled_bits(qc, qubits):
    for i in range(qubits):
        qc.h(i)
        qc.cx(i, i+qubits)

In [91]:
qubits = 10
qc, q, c = set_up_qc(qubits*2)
generate_entangled_bits(qc, qubits)

### Alice generates random bases to decode the key

In [92]:
def generate_random_bits(qubits):
    qc, q, c = set_up_qc(qubits)
    qc.h(q)
    qc.measure(q, c)
    job = execute(qc, backend, shots=1)
    counts = job.result().get_counts()
    bits = list(counts.keys())[0]
    return bits

In [93]:
alice_bases = generate_random_bits(qubits)
print("Alice Bases:", alice_bases)

Alice Bases: 0110000001


### Alice decodes key using bases
```
example:
01000100 - basis
01011010 - key
0?011?10 - result
```
```
basis 1 = Hadamard Basis (Diagonal)
basis 0 = Computational Basis (Recilinear)
```

In [94]:
def decode(qc, bases, start, end):
    gates = ""
    decoded_key = ""
    j = 0
    for i in range(start, end):
        if bases[j] == '1':
            qc.h(i)
            gates += "H"
        else:
            gates += "."
        j += 1
    return gates
    

In [95]:
alice_gates = decode(qc, alice_bases, 0, qubits)
print("Alice Gates Applied:", alice_gates)  

Alice Gates Applied: .HH......H


### Bob generates random bases

In [96]:
bob_bases = generate_random_bits(qubits)
print("Bob Bases:", bob_bases)

Bob Bases: 1110000001


### Bob decodes key using bases

In [97]:
def measure_qc(qc, qubits):
    qc.measure_all()
    job = execute(qc, backend, shots=1)
    counts = job.result().get_counts()
    bits = list(counts.keys())[0][0:qubits*2][::-1]
    return bits

In [98]:
bob_gates = decode(qc, bob_bases, qubits, 2*qubits)
bits = measure_qc(qc, qubits)
alice_decoded_key = bits[0:qubits]
bob_decoded_key = bits[qubits:2*qubits]

print("Alice Decoded Key:", alice_decoded_key)    
print("Bob Decoded Key:", bob_decoded_key)   

print("Bob Gates Applied:", bob_gates)

Alice Decoded Key: 1010111001
Bob Decoded Key: 1010111001
Bob Gates Applied: HHH......H


### Alice shares encoding bases classically and Bob compares bases with Alice

Keys won't match if the quantum states were distrupted due to noise or iterception by Eve. If keys don't match then throw them away and try again.

In [99]:
def find_shared_key(alice_bases, bob_bases, alice_key, bob_decoded_key, qubits):
    alice_shared_key = ""
    bob_shared_key = ""
    for i in range(qubits):
        if alice_bases[i] == bob_bases[i]:
            alice_shared_key += alice_decoded_key[i]
            bob_shared_key += bob_decoded_key[i]
    return alice_shared_key, bob_shared_key
    
        

In [100]:
alice_shared_key, bob_shared_key = find_shared_key(alice_bases, bob_bases, alice_decoded_key, bob_decoded_key, qubits)
print("Alice Shared Key:", alice_shared_key)
print("Bob Shared Key:", bob_shared_key)
if alice_shared_key == bob_shared_key:
    print("Keys match!")
else:
    print("Keys don't match. Discard the keys and run algorithm again")

Alice Shared Key: 010111001
Bob Shared Key: 010111001
Keys match!


### Using more qubits

In [101]:
qubits = 256
print(qubits, "qubit key")
print()

qc, q, c = set_up_qc(qubits*2)
generate_entangled_bits(qc, qubits)


alice_bases = generate_random_bits(qubits)
alice_gates = decode(qc, alice_bases, 0, qubits)
print("Alice Bases:", alice_bases)
print("Alice Gates Applied:", alice_gates)  
print()

bob_bases = generate_random_bits(qubits)
bob_gates = decode(qc, bob_bases, qubits, 2*qubits)
print("Bob Bases:", bob_bases)
print("Bob Gates Applied:", bob_gates)
print()

bits = measure_qc(qc, qubits)
alice_decoded_key = bits[0:qubits]
bob_decoded_key = bits[qubits:2*qubits]

print("Alice Decoded Key:", alice_decoded_key)    
print("Bob Decoded Key:", bob_decoded_key)   
print()

alice_shared_key, bob_shared_key = find_shared_key(alice_bases, bob_bases, alice_decoded_key, bob_decoded_key, qubits)
print("Alice Shared Key:", alice_shared_key)
print("Bob Shared Key:", bob_shared_key)
if alice_shared_key == bob_shared_key:
    print("Keys match!")
else:
    print("Keys don't match. Discard the keys and run algorithm again")
    
print(f'{len(bob_shared_key)} qubit key, {(len(bob_shared_key)/qubits)*100:.2f}% of initial key')
print()
print("Alice and Bob shared key match?:", alice_shared_key==bob_shared_key)

256 qubit key

Alice Bases: 1011011111101110100101101001010000011111100010010000010001001111001000101101101111101101110010111101000000010110110100001011001010010100110011001010100101100110011110001001101000110011100100111110011010011001011001001111100100110100111110010110010100011111
Alice Gates Applied: H.HH.HHHHHH.HHH.H..H.HH.H..H.H.....HHHHHH...H..H.....H...H..HHHH..H...H.HH.HH.HHHHH.HH.HHH..H.HHHH.H.......H.HH.HH.H....H.HH..H.H..H.H..HH..HH..H.H.H..H.HH..HH..HHHH...H..HH.H...HH..HHH..H..HHHHH..HH.H..HH..H.HH..H..HHHHH..H..HH.H..HHHHH..H.HH..H.H...HHHHH

Bob Bases: 1100000100101011001000000000110100110001011110011100110011100001001110011001010011101110001001111101110010110000101111010111100001011011010011010101111100010001000110001000111010000110100110110000000011100101010111110000110111110011000000010111100001010000
Bob Gates Applied: HH.....H..H.H.HH..H.........HH.H..HH...H.HHHH..HHH..HH..HHH....H..HHH..HH..H.H..HHH.HHH...H..HHHHH.HHH..H.HH....H.HHHH.H.HHHH....H.HH.HH.H..HH.H.H.HH